In [1]:
from __future__ import print_function 
from __future__ import division

from dataloader import RetinopathyLoader
from torch.utils import data
from ResNet import *
from run_model import *


import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
from PIL import Image
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.4.0a0+649135b
Torchvision Version:  0.5.0a0


/opt/conda/lib/python3.6/site-packages/torchvision/io/_video_opt.py:17: UserWarning: video reader based on ffmpeg c++ ops not available
  warnings.warn("video reader based on ffmpeg c++ ops not available")


In [2]:
# Batch size for training (change depending on how much memory you have)
batch_size = 4

train_set = RetinopathyLoader(root='./data/',mode='train')
train_loader = data.DataLoader(dataset=train_set,batch_size=batch_size)
test_set = RetinopathyLoader(root='./data/',mode='test')
test_loader = data.DataLoader(dataset=test_set,batch_size=batch_size)

dataloaders_dict = {'train':train_loader,
                    'val' : test_loader}

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

In [5]:
# Number of classes in the dataset
num_classes = 5

# Number of epochs to train for 
num_epochs = 10

# Setup the loss fxn
# Because the data imbalance, we use weight Tensor w to adjust
w = [200,40,20,7,6]
w = [1- (a/sum(w)) for a in w]
w = torch.Tensor(w).to(device)
print(w)
criterion = nn.CrossEntropyLoss(weight=w)

tensor([0.2674, 0.8535, 0.9267, 0.9744, 0.9780], device='cuda:0')


In [6]:
def get_pretrained_model(model_name, num_classes,use_pretrained=True):
    model_dict = {
        'resnet18':  models.resnet18,
        'resnet34':  models.resnet34,
        'resnet50':  models.resnet50,
        'resnet101': models.resnet101,
        'resnet152': models.resnet152
    }
    
    model_ft = model_dict[model_name](pretrained=use_pretrained)          
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(num_ftrs,128)),
        ('relu', nn.ReLU()),
        ('fc2', nn.Linear(128,5))
    ]))
    input_size = 224
    
    return model_ft, input_size

## Run ResNet pretrained model which is from `torchvision`

In [ ]:
# Initialize the pretrained model for this run
model_ft, input_size = get_pretrained_model("resnet18", num_classes=5)
# Send the model to GPU
model_ft = model_ft.to(device)

In [ ]:
# Setup optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=1e-2, momentum=0.9)
lr_sch = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft)

# Train and evaluate pretrained model
pretrained_loss_list, pretrained_acc_train_list, pretrained_acc_test_list, best_acc = \
run(model_ft, dataloaders_dict, criterion, optimizer=optimizer_ft,\
        scheduler=lr_sch,num_epochs=num_epochs)

In [ ]:
torch.save(model_ft,'./models/vision_pretrained_resnet18')
torch.save(pretrained_loss_list,'./result_list/pretrained_loss_list')
torch.save(pretrained_acc_train_list, './result_list/pretrained_acc_train_list')
torch.save(pretrained_acc_test_list, './result_list/pretrained_acc_test_list')

## Run ResNet model (no pretrained) which is from `torchvision`

In [ ]:
# Initialize the non-pretrained version of the model used for this run
#scratch_model,_ = get_pretrained_model(model_name, num_classes=5, use_pretrained=False)
#scratch_model = scratch_model.to(device)
#scratch_loss_list, scratch_acc_train_list, scratch_acc_test_list = \
#    run(scratch_model, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
#torch.save(scratch_model,'./models/vision_not_pretrained_resnet18')

## Run ResNet implemented by myself

In [ ]:
MyResNet18 = ResNet18(num_classes=5)
MyResNet18 = MyResNet18.to(device)

In [ ]:
opt_MyResNet18 = optim.SGD(MyResNet18.parameters(), lr=1e-3,weight_decay=5e-4, momentum=0.9)
lr_sch = optim.lr_scheduler.ReduceLROnPlateau(opt_MyResNet18)

In [ ]:
my_loss_list, my_acc_train_list, my_acc_test_list, best_acc = \
    run(MyResNet18, dataloaders_dict, criterion, optimizer=opt_MyResNet18,\
        scheduler=lr_sch,num_epochs=num_epochs)

In [ ]:
torch.save(MyResNet18,'./models/my_resnet18')
torch.save(my_loss_list,'./result_list/my_loss_list_18')
torch.save(my_acc_train_list, './result_list/my_acc_train_list_18')
torch.save(my_acc_test_list, './result_list/my_acc_test_list_18')

## Comparison

In [ ]:
model_ft = torch.load('./models/vision_pretrained_resnet18')
pretrained_acc_test_list = torch.load('./result_list/pretrained_acc_test_list')

In [ ]:
ohist  = [h.cpu().numpy() for h in pretrained_acc_test_list]
#shist  = [h.cpu().numpy() for h in scratch_acc_test_list]
myhist = [h.cpu().numpy() for h in my_acc_test_list]

plt.figure(figsize=(24,16))
plt.title("Result Comparison(ResNet18)")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
#plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.plot(range(1,num_epochs+1),myhist,label="My model")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

## ResNet 50

In [ ]:
# Initialize the pretrained model for this run
model_ft_50, input_size = get_pretrained_model('resnet50', num_classes=5)
# Send the model to GPU
model_ft_50 = model_ft_50.to(device)

In [ ]:
model_ft_50 = torch.load('./models/vision_pretrained_resnet50')
# Send the model to GPU
model_ft_50 = model_ft_50.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Setup optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9)

# Setup optimizer
optimizer_ft_50 = optim.SGD(model_ft_50.parameters(), lr=1e-3, weight_decay=1e-3, momentum=0.9)
#optimizer_ft_50 = optim.Adam(model_ft_50.parameters(), lr=1e-3, weight_decay=5e-4)
#lr_sch = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft_50)

# Train and evaluate pretrained model
pretrained_loss_list_50, pretrained_acc_train_list_50, pretrained_acc_test_list_50 , best_acc= \
    run(model_ft_50, dataloaders_dict, criterion, optimizer=optimizer_ft_50,\
        num_epochs=10, model_path = './models/vision_pretrained_resnet50', best_acc = 0.)

In [ ]:
best_acc

In [ ]:
#torch.save(model_ft_50,'./models/vision_pretrained_resnet50')
torch.save(pretrained_loss_list_50,'./result_list/pretrained_loss_list_50')
torch.save(pretrained_acc_train_list_50, './result_list/pretrained_acc_train_list_50')
torch.save(pretrained_acc_test_list_50, './result_list/pretrained_acc_test_list_50')

In [ ]:
MyResNet50 = ResNet50(num_classes=5)
MyResNet50 = MyResNet50.to(device)
my_loss_list_50, my_acc_train_list_50, my_acc_test_list_50, best_acc = \
    run(MyResNet50, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(MyResNet50,'./models/my_resnet50')
torch.save(my_loss_list_50,'./result_list/my_loss_list_50')
torch.save(my_acc_train_list_50, './result_list/my_acc_train_list_50')
torch.save(my_acc_test_list_50, './result_list/my_acc_test_list_50')

In [ ]:
MyResNet101 = ResNet101(num_classes=5)
MyResNet101 = MyResNet101.to(device)
my_loss_list_101, my_acc_train_list_101, my_acc_test_list_101, best_acc = \
    run(MyResNet101, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(MyResNet101,'./models/my_resnet101')
torch.save(my_loss_list_101,'./result_list/my_loss_list_101')
torch.save(my_acc_train_list_101, './result_list/my_acc_train_list_101')
torch.save(my_acc_test_list_101, './result_list/my_acc_test_list_101')

# Run ResNet152

### With pretrained resnet 152

In [7]:
pretrained_152 ,_ = get_pretrained_model(model_name='resnet152',num_classes=5)
pretrained_152 = pretrained_152.to(device)
criterion = nn.CrossEntropyLoss()

In [12]:
pretrained_152 = torch.load('./models/vision_pretrained_152')
pretrained_152 = pretrained_152.to(device)

In [16]:
opt_pretrained_152 = optim.SGD(pretrained_152.parameters(), lr=1e-3,weight_decay=5e-3, momentum=0.9)
lr_sch = optim.lr_scheduler.ReduceLROnPlateau(opt_pretrained_152)

In [ ]:
pretrained_loss_list_152, pretrained_acc_train_list_152, pretrained_acc_test_list_152, best_acc = \
    run(pretrained_152, dataloaders_dict, criterion,optimizer=opt_pretrained_152\
        , scheduler=lr_sch, num_epochs=20, model_path = './models/vision_pretrained_152', best_acc = 0.7805)

Epoch 1/20
----------
train Loss: 0.5592 Acc: 0.8085
train pred_list.value_counts() = 
0    23967
2     3381
4      438
3      295
1       18
dtype: int64
val Loss: 0.7146 Acc: 0.7640
val pred_list.value_counts() = 
0    5883
2     847
3     163
4     132
dtype: int64
Epoch 2/20
----------
train Loss: 0.6342 Acc: 0.7844
train pred_list.value_counts() = 
0    24696
2     2824
4      325
3      253
1        1
dtype: int64
val Loss: 0.7671 Acc: 0.7476
val pred_list.value_counts() = 
0    5639
2    1208
3     122
4      56
dtype: int64
Epoch 3/20
----------
train Loss: 0.6889 Acc: 0.7663
train pred_list.value_counts() = 
0    25410
2     2266
4      256
3      167
dtype: int64
val Loss: 0.7327 Acc: 0.7613
val pred_list.value_counts() = 
0    6444
2     501
3      47
4      33
dtype: int64
Epoch 4/20
----------
train Loss: 0.7222 Acc: 0.7574
train pred_list.value_counts() = 
0    25933
2     1887
4      152
3      127
dtype: int64
val Loss: 0.7382 Acc: 0.7589
val pred_list.value_counts() = 

In [27]:
best_acc

tensor(0.7611, device='cuda:0', dtype=torch.float64)

In [14]:
#torch.save(pretrained_152,'./models/vision_pretrained_152')
torch.save(pretrained_loss_list_152,'./result_list/pretrained_loss_list_152')
torch.save(pretrained_acc_train_list_152, './result_list/pretrained_acc_train_list_152')
torch.save(pretrained_acc_test_list_152, './result_list/pretrained_acc_test_list_152')

In [ ]:
MyResNet152 = ResNet152(num_classes=5)
opt_MyResNet152 = optim.SGD(MyResNet152.parameters(), lr=1e-2,weight_decay=5e-3, momentum=0.9)
lr_sch = optim.lr_scheduler.ReduceLROnPlateau(opt_MyResNet152)
MyResNet152 = MyResNet152.to(device)
#print(MyResNet152)

In [ ]:
my_loss_list_152, my_acc_train_list_152, my_acc_test_list_152 , best_acc= \
    run(MyResNet152, dataloaders_dict, criterion, optimizer=opt_MyResNet152,scheduler=lr_sch,\
        num_epochs=5)

In [ ]:
torch.save(MyResNet152,'./models/my_resnet152')
torch.save(my_loss_list_152,'./result_list/my_loss_list_152')
torch.save(my_acc_train_list_152, './result_list/my_acc_train_list_152')
torch.save(my_acc_test_list_152, './result_list/my_acc_test_list_152')